In [2]:
# Cell 1: Setup environment and imports
import os
import sys
import time
import numpy as np
import json

# Add project directory to path
sys.path.append(os.path.dirname(os.path.abspath('__file__')))

# Set environment variables for display over SSH
os.environ['DISPLAY'] = ':0'
os.environ['QT_DEBUG_PLUGINS'] = '1'
os.environ['QT_QPA_PLATFORM_PLUGIN_PATH'] = '/usr/lib/aarch64-linux-gnu/qt5/plugins/platforms'

print("Environment setup complete")


Environment setup complete


In [4]:
# Cell 2: System Initialization
from scanning_system import ScanningSystem
from arm import DualArmController
from turntable_control import StepperMotorManager
from viz import ArmVisualizer

# Initialize system components
system = ScanningSystem('/dev/ttyUSB0', '/dev/ttyUSB1')  # Adjust ports as needed
motor_manager = StepperMotorManager()

print("System components initialized")


ModuleNotFoundError: No module named 'scanning_system'

In [ ]:
# Cell 3: Arm Movement Testing
# Connect to arms
if system.controller.connect():
    print("Arms connected successfully")
else:
    print("Failed to connect to arms")

# Define safe boundaries (in mm)
SAFE_BOUNDARIES = {
    'x': (-350, 350),
    'y': (-350, 350),
    'z': (50, 350)
}

def move_arm_safely(controller, arm_side, x, y, z):
    """Move arm within safe boundaries"""
    if (SAFE_BOUNDARIES['x'][0] <= x <= SAFE_BOUNDARIES['x'][1] and
        SAFE_BOUNDARIES['y'][0] <= y <= SAFE_BOUNDARIES['y'][1] and
        SAFE_BOUNDARIES['z'][0] <= z <= SAFE_BOUNDARIES['z'][1]):

        if arm_side == 'left':
            controller.move_left_to_world(x, y, z)
        elif arm_side == 'right':
            controller.move_right_to_world(x, y, z)
        else:
            raise ValueError("arm_side must be 'left' or 'right'")

        print(f"Moved {arm_side} arm to ({x}, {y}, {z})")
    else:
        print(f"Position ({x}, {y}, {z}) is outside safe boundaries")

# Test movements
move_arm_safely(system.controller, 'left', 200, -100, 150)
move_arm_safely(system.controller, 'right', 200, 100, 150)
time.sleep(2)

# Move back to safe positions
move_arm_safely(system.controller, 'left', 300, -200, 100)
move_arm_safely(system.controller, 'right', 300, 200, 100)


In [ ]:
# Cell 4: Gripper Control Testing
# Test gripper functions
print("Testing gripper functions...")

# Enable torque lock
system.controller.left_arm.torque_lock(True)
system.controller.right_arm.torque_lock(True)
print("Torque lock enabled for both arms")
time.sleep(1)

# Test gripper movements
system.controller.grasp_with_left()
print("Left gripper closed")
time.sleep(1)

system.controller.release_with_left()
print("Left gripper opened")
time.sleep(1)

system.controller.grasp_with_right()
print("Right gripper closed")
time.sleep(1)

system.controller.release_with_right()
print("Right gripper opened")
time.sleep(1)

# Disable torque lock
system.controller.left_arm.torque_lock(False)
system.controller.right_arm.torque_lock(False)
print("Torque lock disabled for both arms")


In [ ]:
# Cell 5: Turntable Control Testing
# Test turntable motor
print("Testing turntable motor...")

# Rotate turntable 90 degrees
motor_manager.set_motor_state("moving", "move_by", 90)
print("Turntable rotating 90 degrees")
time.sleep(2)

# Rotate back to 0 degrees
motor_manager.set_motor_state("moving", "move_to", 0)
print("Turntable returning to 0 degrees")
time.sleep(2)

# Stop motor
motor_manager.set_motor_stopped()
print("Turntable stopped")


In [ ]:
# Cell 6: Coordinate System Calibration
# Calibration points around sample center
calibration_points = [
    (0, 0, 100),      # Above sample
    (100, 0, 0),      # Right of sample
    (-100, 0, 0),     # Left of sample
    (0, 100, 0),      # Front of sample
    (0, -100, 0),     # Back of sample
    (70, 70, 50),     # Diagonal front-right
    (-70, 70, 50),    # Diagonal front-left
]

print("Starting calibration process...")
try:
    system.controller.calibrate_arms(calibration_points)
    system.is_calibrated = True
    print("Calibration completed successfully")

    # Save calibration
    system.save_calibration("calibration.pkl")
    print("Calibration saved to file")
except Exception as e:
    print(f"Calibration failed: {e}")


In [ ]:
# Cell 7: Visualization Testing
# Test visualization
print("Testing visualization...")

# Initialize visualizer
system.visualizer = ArmVisualizer(system.sample_center, system.arm_bases)

# Plan a simple scan
position_pairs = system.plan_scan(
    r_range=(100, 200),
    theta_range=(0, np.pi/4),
    phi_range=(0, np.pi),
    r_steps=3,
    theta_steps=2,
    phi_steps=4
)

print(f"Planned {len(position_pairs)} scanning positions")

# Visualize the scan path
system.visualize_plan(position_pairs)

# Show visualization window
system.visualizer.show()


In [ ]:
# Cell 8: Spectrometer Integration
# Spectrometer testing
print("Testing spectrometer integration...")

try:
    # Import spectrometer (if capture.py is properly structured)
    from capture import Spectrometer

    # Initialize spectrometer
    spectrometer = Spectrometer(camera_index=0)
    print("Spectrometer initialized")

    # Capture a frame
    frame = spectrometer.capture_frame()
    if frame is not None:
        print("Frame captured successfully")

        # Extract profile
        profile = spectrometer.extract_profile(frame)
        if profile is not None:
            print(f"Profile extracted with shape: {profile.shape}")
        else:
            print("Failed to extract profile")
    else:
        print("Failed to capture frame")

except ImportError as e:
    print(f"Spectrometer import failed: {e}")
    print("Make sure capture.py is properly configured")
except Exception as e:
    print(f"Spectrometer test failed: {e}")


In [ ]:
# Cell 9: Simple Scan Execution
# Execute a simple scan
print("Executing simple scan...")

# Plan a minimal scan
position_pairs = system.plan_scan(
    r_range=(100, 150),
    theta_range=(0, 0.3),
    phi_range=(0, 1.5),
    r_steps=2,
    theta_steps=2,
    phi_steps=3
)

print(f"Planned {len(position_pairs)} positions")

# Execute scan (without turntable for simplicity)
system.execute_scan(position_pairs[:5], use_turntable=False)

print("Simple scan execution completed")


In [ ]:
# Cell 10: Data Saving and Visualization
# Save scan data
import json
from datetime import datetime

print("Saving scan data...")

# Example data structure
scan_data = {
    "timestamp": datetime.now().isoformat(),
    "positions": [],
    "measurements": []
}

# Add some dummy data
for i in range(5):
    scan_data["positions"].append({
        "left": [200, -100+i*10, 150],
        "right": [200, 100-i*10, 150]
    })
    scan_data["measurements"].append({
        "wavelength": list(range(400, 700, 10)),
        "intensity": list(np.random.rand(30))
    })

# Save to file
filename = f"scan_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(filename, 'w') as f:
    json.dump(scan_data, f, indent=2)

print(f"Scan data saved to {filename}")

# Basic visualization of saved data
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))

# Plot positions
plt.subplot(1, 2, 1)
left_x = [pos["left"][0] for pos in scan_data["positions"]]
left_y = [pos["left"][1] for pos in scan_data["positions"]]
plt.scatter(left_x, left_y, c='blue', label='Left Arm')
plt.scatter([0], [0], c='red', s=100, label='Sample')
plt.xlabel('X (mm)')
plt.ylabel('Y (mm)')
plt.title('Scanning Positions')
plt.legend()
plt.grid(True)

# Plot sample spectrum
plt.subplot(1, 2, 2)
wavelengths = scan_data["measurements"][0]["wavelength"]
intensities = scan_data["measurements"][0]["intensity"]
plt.plot(wavelengths, intensities, 'g-')
plt.xlabel('Wavelength (nm)')
plt.ylabel('Intensity')
plt.title('Sample Spectrum')
plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
# Cell 11: System Cleanup
# Cleanup system resources
print("Cleaning up system resources...")

try:
    # Release instruments
    system.release_instruments()

    # Disconnect from arms
    system.controller.disconnect()

    # Cleanup motor manager
    motor_manager.cleanup()

    # Cleanup LED manager
    system.led_manager.cleanup()

    print("System cleanup completed successfully")

except Exception as e:
    print(f"Error during cleanup: {e}")
